<a href="https://colab.research.google.com/github/minsuk-heo/tf2/blob/master/jupyter_notebooks/01.Node.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Node 만들기

In [1]:
# For Colab to use Tensorflow 2.X
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass

In [2]:
import tensorflow as tf

In [3]:
tf.__version__

'2.7.0'

## 함수 없이 구현 하기

In [22]:
# one feature node
x = tf.constant([[1.0]])

# weight, bias
w = tf.Variable([[2.0]])
b = tf.Variable([[1.0]])

In [25]:
node = tf.add(tf.matmul(x,w), b)
print(node.numpy())

[[3.]]


In [26]:
#sigmoid
out = tf.math.sigmoid(node)
print(out.numpy())

[[0.95257413]]


## two feature node

In [31]:
w = tf.Variable([[1.1], [2.2]])
b = tf.Variable([[-1.0]])
print(w.numpy())

[[1.1]
 [2.2]]


In [37]:
x = tf.constant([[2.0, -1.0]])
# x = tf.constant([[2.0],[ -1.0]])
print(x.numpy())
print(x.shape)

[[ 2. -1.]]
(1, 2)


In [38]:
node = tf.add(tf.matmul(x, w), b)
out = tf.math.sigmoid(node)
print(out.numpy())


[[0.26894143]]


In [41]:
## multi input data
x = tf.constant([[1.0, 2.0], [-1.0, 1.0], [2.0, 2.0]])
print(x.shape)

(3, 2)


In [42]:
node = tf.add(tf.matmul(x, w), b)
out = tf.math.sigmoid(node)
print(out.numpy())


[[0.9890131 ]
 [0.52497923]
 [0.9963158 ]]


# One Feature Node

In [4]:
class One_Feature_Node:
    def __init__(self):
        self.w = tf.Variable([[0.1]])
        self.b = tf.Variable([[0.5]])
        
    def __call__(self, x):
        return self.get_output(x)
    
    def get_output(self,x):
        out = tf.matmul(x,self.w)
        print("matmul result: ", out.numpy())
        out = tf.add(out, self.b)
        print("adding bias result: ", out.numpy())
        out = tf.math.sigmoid(out)
        print("sigmoid result:", out.numpy())
        return out

In [5]:
# Single Input
x = tf.constant([[1.0]])

one_feature_node = One_Feature_Node()
one_feature_node(x)

2022-02-14 20:47:25.917771: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-14 20:47:25.918013: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

matmul result:  [[0.1]]
adding bias result:  [[0.6]]
sigmoid result: [[0.6456563]]


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.6456563]], dtype=float32)>

# Two Features Node

In [6]:
class Two_Features_Node:
    def __init__(self):
        self.w = tf.Variable([[0.1], [0.2]])
        self.b = tf.Variable([[0.5]])
        
    def __call__(self, x):
        return self.get_output(x)
    
    def get_output(self,x):
        out = tf.matmul(x,self.w)
        print("matmul result: ", out.numpy())
        out = tf.add(out, self.b)
        print("adding bias result: ", out.numpy())
        out = tf.math.sigmoid(out)
        print("sigmoid result:", out.numpy())
        return out

In [7]:
# Multi Features Single Input (Two features for one data)
x = tf.constant([[1.0,2.0]])

two_features_node = Two_Features_Node()
two_features_node(x)

matmul result:  [[0.5]]
adding bias result:  [[1.]]
sigmoid result: [[0.7310586]]


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.7310586]], dtype=float32)>

# N inputs for Two Features Node

In [8]:
# Multi Input Data (3 samples)
x = tf.constant([[1.0,2.0],[2.0,3.0],[4.0,5.0]])

two_features_node = Two_Features_Node()
two_features_node(x)

matmul result:  [[0.5]
 [0.8]
 [1.4]]
adding bias result:  [[1. ]
 [1.3]
 [1.9]]
sigmoid result: [[0.7310586]
 [0.785835 ]
 [0.8698916]]


<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[0.7310586],
       [0.785835 ],
       [0.8698916]], dtype=float32)>

# tf.function speed check
tf.function decoration put your python onto autograph.  
This enables faster execution, running on GPU or TPU, or exporting to SavedModel.

In [9]:
import timeit

In [12]:
class Two_Features_Node2:
    def __init__(self):
        self.w = tf.Variable([[0.1], [0.2]])
        self.b = tf.Variable([[0.5]])
        
    def __call__(self, x):
        return self.get_output(x)
    
    def get_output(self,x):
        for i in range(50):
            out = tf.matmul(x,self.w)
            out = tf.add(out, self.b)
            out = tf.math.sigmoid(out)
        return out
    
    @tf.function
    def get_output_faster(self,x):
        for i in range(50):
            out = tf.matmul(x,self.w)
            out = tf.add(out, self.b)
            out = tf.math.sigmoid(out)
        return out

In [13]:
two_features_node2 = Two_Features_Node2()
get_output_without_tf_function_timeit = timeit.timeit(lambda: two_features_node2.get_output(x), number=300)
get_output_faster_with_tf_function_timeit = timeit.timeit(lambda: two_features_node2.get_output_faster(x), number=300)
time_diff = round(get_output_without_tf_function_timeit / get_output_faster_with_tf_function_timeit, 1)

2022-02-14 20:49:34.241850: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-02-14 20:49:34.242021: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [14]:
print("get_output_faster(@tf.function) is " + str(time_diff) + " times faster than get_output_without_tf_function!")

get_output_faster(@tf.function) is 11.0 times faster than get_output_without_tf_function!
